# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# collect each filepath for csv file
file_path_list = glob.glob(filepath + "/*.csv")
print(len(file_path_list))

/home/workspace
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
 
 #
# get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_new.csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# create a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace sparkifydb
keyspace_create = ("""
CREATE KEYSPACE IF NOT EXISTS sparkifydb
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor' : 1}
"""
)

try:
    session.execute(keyspace_create)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# connect to keyspace sparkifydb
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


## Query 1

### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4 

To answer this question, we should do the following steps:
1. Analyze this query requirement to get query statement as below: <br>
SELECT artist, song_title, song_len FROM session_item WHERE session_id=338 and item_in_session=4
2. Design and create a table in keyspace 'sparkifydb' based on the above query statement
    * Create table named session_item
    * The table should include columns: session_id, item_in_session, artist, song_title and song_len  
    * Partition Key is session_id, clustering column is item_in_session
3. Load data from 'event_datafile_new.csv' into this session_item table
4. Execute SELECT statement to verify the data was entered into the table

In [8]:
# create table statement
create_session_item_table = ("""
CREATE TABLE IF NOT EXISTS session_item (
    session_id int,
    item_in_session int,
    artist text,
    song_title text,
    song_len float,
    PRIMARY KEY((session_id), item_in_session))
""")

# create table in Apache Cassandra
session.execute(create_session_item_table)                    

In [9]:
# insert data into table from csv file
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements
        query = "INSERT INTO session_item (session_id,item_in_session, artist, song_title, song_len)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into session_item table

In [10]:
query = "SELECT artist, song_title, song_len FROM session_item WHERE session_id=338 and item_in_session=4"

rows = session.execute(query)
for row in rows:
    print(row.artist, ",", row.song_title, ",", row.song_len)

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


## Query 2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question, we should do the following steps:
1. Analyze this query requirement to get query statement as below: <br>
SELECT artist, song_title, first_name, last_name FROM user_session WHERE user_id = 10 and session_id=182
2. Design and create a table in keyspace 'sparkifydb' based on the above query statement
    * Create table named user_session
    * The table should include columns: user_id,session_id, item_in_session, first_name, last_name, artist, song_title  
    * Partition Key is user_id and session_id , clustering column is item_in_session
3. Load data from 'event_datafile_new.csv' into this user_session table
4. Execute SELECT statement to verify the data was entered into the table

In [11]:
# create table statement
create_user_session_table = ("""
CREATE TABLE IF NOT EXISTS user_session (
    user_id int,
    session_id int,
    item_in_session int,
    first_name text,
    last_name text,
    artist text,
    song_title text,
    PRIMARY KEY((user_id, session_id), item_in_session))
""")

# create table in Apache Cassandra
session.execute(create_user_session_table)             

In [12]:
#  insert data into user_session table from csv file
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (user_id, session_id, item_in_session, first_name, last_name, artist, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4],line[0], line[9]))

In [13]:
#execute SELECT statement to verify the data was entered into the table
query = "SELECT artist, song_title, first_name, last_name FROM user_session WHERE user_id = 10 and session_id=182"
rows = session.execute(query)
for row in rows:
    print(row.artist, ",", row.song_title, ",", row.first_name, row.last_name)

Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


## Query 3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer this question, we should do the following steps:
1. Analyze this query requirement to get query statement as below: <br>
SELECT first_name, last_name FROM song_user WHERE song_title='All Hands Against His Own'
2. Design and create a table in keyspace 'sparkifydb' based on the above query statement
    * Create table named song_user
    * The table should include columns: song_title,user_id,first_name,last_name 
    * Partition Key is song_title, clustering column is user_id
3. Load data from 'event_datafile_new.csv' into this song_user table
4. Execute SELECT statement to verify the data was entered into the table

In [14]:
# create table statement
create_song_user_table = ("""
CREATE TABLE IF NOT EXISTS song_user (
    song_title text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY((song_title), user_id))
""")

# create table in Apache Cassandra
session.execute(create_song_user_table)  

In [15]:
# insert data into song_user table from csv file
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# execute Select statement to verify the data was entered into the table
query = "SELECT first_name, last_name FROM song_user WHERE song_title='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop table session_item if exists for query 1
session.execute("DROP TABLE IF EXISTS session_item")

# Drop table user_session if exists for query 2
session.execute("DROP TABLE IF EXISTS user_session")

# Drop table song_user if exists for query 3
session.execute("DROP TABLE IF EXISTS song_user")

### Close the session and cluster connection¶

In [18]:
# close session
session.shutdown()

# close cluster connection
cluster.shutdown()